# Project Idea 2:
- A good loan (from the prospective of an investor) pays 
the interests and fractional principals on time and terminate at loan maturity.
- An investor often lose money when a loan goes into default, settlement,
or 'written off' (called **charged off** in this data set).
- Build a supervised model to make **multi-label** prediction on 3 dimensions
"charged off + default", "settlement involved", "hardship".
- This can be used either by **Lending Club** itself or a third-party investing firm
for loan-grade design or accurated portfolio selection.
- Depending on the scope of your project, you may 
    - tackle a single label prediction.
    - restrict to the pooled models.
    - focus on the time seris models
- This is a **multi-label** binary imbalance classification task.
- If you train a **pooled** model, you have to deal with $2M+$ samples, often too
large for a typical ML algorithm to handle.

- Try several imbalance classification techniques and evaluate their performance.

- Based on your business, discuss the negative impacts of type I (false
positive), type II (false negative) errors in your prediction.

- If you decide to train a time series model, make sure that you have some
basic background on performing hyper-parameter tuning in the time series context.

- **MUST**: A defaulted loan with a loan amount $\$1000$ has a totally different 
impact to the final profit than a defaulted $\$50000$ loan. 
   - Discuss whether the **classroom-taught** machine learning techniques 
    addresses these issues. How would you modify the classifier to take into account 
         - your business objectives.
         - the profit and loss focus.

- Can you use **NLP** technique to extract insights on the loan descriptions
which helps your predictive task?
</a><br>
# Structure: 
- <a href="#preprocessing">Preprocessing</a><br>
    - <a href="#before">Application Information before loan issued</a><br>
    - <a href="#label">Lables</a><br>
    - <a href="#missing">Missing Values</a><br>
- <a href="#function">Function</a><br>
- <a href="#ml">Machine Learning</a><br>
    - Unsupervised Machine Learning  
        - <a href="#kmeans">K Means</a><br> 

    - Supervised Machine Learning      
        - <a href="#decision">Decision Tree</a><br>
        - <a href="#rf">Random Forest</a><br>
        - <a href="#svm">SVM</a><br>
        - <a href="#xgboost">XGBoost</a><br>
        - <a href="#logistic">Logistic Regression</a><br>
        - <a href="#naive">Naive Bayes Classifier</a><br>
        - <a href="#neighbor">Nearest Neighbor</a><br>
- <a href="#imbalance">Handling Imbalanced Data</a><br>
    - <a href="#smote">SMOTE</a><br>

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

import seaborn as sns
sns.set(style="whitegrid")
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib as matplotlib
%matplotlib inline

In [ ]:
df_raw_accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')

In [ ]:
df_raw_accepted.sample(frac=0.001).to_csv('sample_accepted.csv')

In [ ]:
sample_accepted = pd.read_csv('sample_accepted.csv')

In [ ]:
sample_accepted.shape

In [ ]:
sample_accepted = sample_accepted.sample(2000)

 <p><a name="preprocessing"></a></p>
 
 ## Preprocessing

In [ ]:
df_processed = sample_accepted.copy()

 <p><a name="before"></a></p>
 
 ### Application Information before loan is issued 

In [ ]:
# Secondary Applicant

joint_list = ['sec_app_fico_range_low','sec_app_fico_range_high',
               'sec_app_earliest_cr_line','sec_app_inq_last_6mths',
               'sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util',
               'sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths',
               'sec_app_collections_12_mths_ex_med','sec_app_mths_since_last_major_derog',
               'verification_status_joint','revol_bal_joint', 
               'dti_joint', 'application_type','annual_inc_joint']

In [ ]:
df_processed.loc[df_processed['application_type'] == 'Joint App',joint_list]

 <p><a name="label"></a></p>
 
 ### Lables

In [ ]:
# Multi Labels - "charged off + default", "settlement involved", "hardship" 

label_list = ['label_hardship','label_chargedoff_default','label_settlement', 'loan_status' ]
chargedoff_default_list = ['Charged Off','Default', 'Does not meet the credit policy. Status:Charged Off']

df_processed['label_hardship'] = df_processed['hardship_status'].apply(lambda x: 0 if x  is np.nan else 1)
df_processed['label_chargedoff_default'] = df_processed['loan_status'].apply(lambda x: 1 if x in chargedoff_default_list else 0)
df_processed['label_settlement'] = df_processed['settlement_status'].apply(lambda x: 0 if x  is np.nan else 1)

In [ ]:
# Hardship Loans 

hardship_list = ['hardship_flag',
     'hardship_type',
     'hardship_reason',
     'hardship_status',
     'deferral_term',
     'hardship_amount',
     'hardship_start_date',
     'hardship_end_date',
     'payment_plan_start_date',
     'hardship_length',
     'hardship_dpd',
     'hardship_loan_status',
     'orig_projected_additional_accrued_interest',
     'hardship_payoff_balance_amount',
     'hardship_last_payment_amount',
     'label_hardship'
                ]

In [ ]:
df_processed.loan_status.unique()

In [ ]:
df_processed[hardship_list][df_processed['hardship_type'].notnull()]

In [ ]:
# Settlement Loans 

settlement_list = ['debt_settlement_flag',
     'debt_settlement_flag_date',
     'settlement_status',
     'settlement_date',
     'settlement_amount',
     'settlement_percentage',
     'settlement_term']

In [ ]:
df_processed[settlement_list][df_processed['settlement_status'].notnull()]

In [ ]:
df_label = df_processed[label_list].copy()
df_label['total'] = df_label['label_hardship'] + df_label['label_chargedoff_default'] + df_label['label_settlement']
# df_label.loc[df_label['label_settlement'] == 1].sample(10)
df_label.groupby(['total', 'label_hardship', 'label_settlement' , 'label_chargedoff_default']).size()

In [ ]:
df_label.loc[df_label['label_settlement'] == 1].groupby('loan_status').size()

In [ ]:
df_label.loc[df_label['label_hardship'] == 1].groupby('loan_status').size()

In [ ]:
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
    labels=["Charged Off", "Current", "Late", "Hardship", "Hardship", "Hardship","Settlement","Settlement"],
    parents=["Total", "Total", "Total", "Charged Off", "Current", "Late", "Charged Off", "Late" ],
    values=[980, 235, 743, 42, 5, 1, 3, 32, 1],
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

# https://plot.ly/python/sunburst-charts/

 <p><a name="missing"></a></p>
    
### Missing Values 

In [ ]:
total = df_processed.isnull().sum().sort_values(ascending=False)
percent = (df_processed.isnull().sum()/df_processed.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(100)

In [ ]:
# Drop irrelavant columns 
drop_list = ['Unnamed: 0','id','member_id','funded_amnt','url','desc','title']

drop_for_grade_list = ['funded_amnt_inv','int_rate','installment','issue_d','loan_status','pymnt_plan','out_prncp','out_prncp_inv']

df_processed = df_processed.drop(drop_list, axis=1)
df_processed = df_processed.drop(drop_for_grade_list, axis=1)

# Convert categorical to numerical 
df_processed['term'] = df_processed['term'].apply(lambda x: int(x.split()[0]))
df_processed['emp_length'] = df_processed['emp_length'].str.extract('(\d+)') 
#10 means more than 10 years 

# Convert to Datetime
df_processed['earliest_cr_line'] = pd.to_datetime(df_processed['earliest_cr_line'])

# Missing Values 

df_processed.mths_since_last_record = df_processed.mths_since_last_record.fillna(0)
df_processed.mths_since_last_delinq = df_processed.mths_since_last_delinq.fillna(0)

df_processed.emp_title = df_processed.emp_title.fillna('None')
df_processed.emp_length = df_processed.emp_length.fillna(0)

df_processed.revol_util = df_processed.revol_util.fillna(0)

df_processed.dti = df_processed.dti.fillna(df_processed.revol_bal / df_processed.annual_inc)